# Roboto SDK - Analyze 1,000 PX4 Flights

This notebook shows how to use the [Roboto SDK](https://docs.roboto.ai/learn/sdk.html) to aggregate metrics from a large number of log files and analyze the results.

We'll look at a couple of examples, where we’ll role-play different job functions in a company building an autonomous drone with the PX4 Flight Controller. 

The data used in this notebook is public and from the PX4 Flight Review website. It comprises 1,000 flights, downloaded in November 2024. You can see the [collection](https://app.roboto.ai/collections/cl_ms5mbx5dww76) on Roboto.

## Initialization

In [ ]:
import roboto
from statistics import mean
from notebook_utils.px4_results import plot_indicators, create_bar_chart, get_countries_by_reverse_gps

query_client = roboto.query.QueryClient(
    owner_org_id="og_ghla0xxceeg8"  # PX4 Demo (public)
)

roboto_search = roboto.RobotoSearch(query_client)

## (1) Altitude incursions, geofence violations and RC signal loss

As a Systems Engineer, your task is to report on key operational metrics, including altitude incursions—instances where the drone exceeded the maximum flight altitude of 250m—geofence violations, where the drone left the designated flight area, and occurrences of remote control (RC) signal loss.

In [ ]:
query = 'topics[0].msgpaths[vehicle_air_data.baro_alt_meter].max > 250 AND created > "2024-01-01"'
results = roboto_search.find_files(query)
nr_altitude_incursion_events = len(list(results))

In [ ]:
query = 'topics[0].msgpaths[vehicle_status.geofence_violated].true_count > 0 AND created > "2024-01-01"'
results = roboto_search.find_files(query)
nr_geofence_violations = len(list(results))

In [ ]:
query = 'topics[0].msgpaths[vehicle_status.rc_signal_lost].true_count > 0 AND created > "2024-01-01"'
results = roboto_search.find_files(query)
nr_rc_signal_lost = len(list(results))

In [ ]:
plot_indicators([
    {"title": "Altitude Incursions 2024", "value": nr_altitude_incursion_events},
    {"title": "Geofence Violations 2024", "value": nr_geofence_violations},
    {"title": "RC Signal Lost 2024", "value": nr_rc_signal_lost}
])

## (2) CPU load statistics for master software branch
As an Avionics Engineer developing the next-generation autopilot, you need to analyze CPU load metrics from flights using the `master` branch. Key metrics include:

- Min CPU Load
- Max CPU Load
- Mean CPU Load

In [ ]:
query = 'topic.name = "cpuload" AND path="load" AND file.metadata.ver_sw_branch = "master"'
results = list(roboto_search.find_message_paths(query))

min_cpu_load_master_branch = min([m.min for m in results])*100
max_cpu_load_master_branch = max([m.max for m in results])*100
mean_cpu_load_master_branch = mean([m.mean for m in results])*100

In [ ]:
plot_indicators([
    {"title": "Min CPU Load", "value": min_cpu_load_master_branch, "suffix": "%"},
    {"title": "Max CPU Load", "value": max_cpu_load_master_branch, "suffix": "%"},
    {"title": "Mean CPU Load", "value": mean_cpu_load_master_branch, "suffix": "%"}
])

## (3) Compare the mean accelerometer temperature between different hardware versions

As a Thermal Engineer, you need to investigate reports of accelerometer overheating following a hardware upgrade. Specifically, you need to retrieve the mean accelerometer temperature for hardware versions `PX4_FMU_V6C` and `PX4_FMU_V3`.

In [ ]:
query1 = 'topic.name = "vehicle_imu_status_00" AND path="temperature_accel" AND file.metadata.ver_hw = "PX4_FMU_V6C"'
query2 = 'topic.name = "vehicle_imu_status_00" AND path="temperature_accel" AND file.metadata.ver_hw = "PX4_FMU_V3"'

results1 = list(roboto_search.find_message_paths(query1))
results2 = list(roboto_search.find_message_paths(query2))

mean_accelerometer_temp_px4_fmu_v6c = mean([m.mean for m in results1])
mean_accelerometer_temp_px4_fmu_v3 = mean([m.mean for m in results2])

In [ ]:
plot_indicators([
    {"title": "Mean Accelerometer Temp PX4_FMU_V6C", "value": mean_accelerometer_temp_px4_fmu_v6c, "suffix": "°C"},
    {"title": "Mean Accelerometer Temp PX4_FMU_V3", "value": mean_accelerometer_temp_px4_fmu_v3, "suffix": "°C"},
])

## (4) Analyze flight distribution by country and popularity of PX4 hardware versions

Finally, we thought it would be interesting to identify the most popular PX4 hardware versions used across flights from last year, and the distribution of drone flights by country. 

Let’s start with the breakdown of PX4 hardware versions:

In [ ]:
from collections import Counter

# Retrieve and count `ver_hw` occurrences
results = roboto_search.find_files("*")
ver_hw_counts = Counter(entry.metadata.get('ver_hw') for entry in results if entry.metadata.get('ver_hw'))

# Sort counts in descending order
sorted_ver_hw_counts = dict(sorted(ver_hw_counts.items(), key=lambda item: item[1], reverse=True))

# Extract hardware names and values
hardware_names = list(sorted_ver_hw_counts.keys())
hardware_values = list(sorted_ver_hw_counts.values())

create_bar_chart(x=hardware_names[:20], y=hardware_values[:20], title="Number of Flights by Hardware Version", xaxis_title="Hardware Version", yaxis_title="Count")

Now let’s try to find the distribution of drone flights by country. 

To achieve this, we’ll start by obtaining the median latitude and longitude for each flight that includes the `vehicle_gps_position` topic. Next, we’ll perform a reverse GPS lookup with a free API to determine the country for each set of coordinates.

In [ ]:
# The field names in the `vehicle_gps_position` topic vary by PX4 version: 
# It may be lat/lon or latitude_deg/longitude_deg; we query both below.

# Helper function to fetch query results and normalize values
def get_coordinates(query):
    normalization_factors = {"lat": 1e7, "lon": 1e7}
    result_list = roboto_search.find_message_paths(query)
    return [
        entry.mean / normalization_factors.get(entry.path, 1)
        for entry in result_list
    ]

# Define query strings to obtain latitude and longitude values
latitude_query = 'topic.name="vehicle_gps_position" AND (path="lat" OR path="latitude_deg")'
longitude_query = 'topic.name="vehicle_gps_position" AND (path="lon" OR path="longitude_deg")'

# Retrieve latitude and longitude lists
latitude_list = get_coordinates(latitude_query)
longitude_list = get_coordinates(longitude_query)

# Lookup countries from GPS coordinates (this may take some time)
country_dict = get_countries_by_reverse_gps(latitude_list, longitude_list)

# Sort and extract country names and values
sorted_country_dict = dict(sorted(country_dict.items(), key=lambda item: item[1], reverse=True))

create_bar_chart(x=list(sorted_country_dict.keys()), y=list(sorted_country_dict.values()), title="Number of Flights by Country", xaxis_title="Countries", yaxis_title="Count")